In [17]:
# Pickle
import pickle
# Pandas
import pandas as pd
# Hugging Face
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# PyTorch
import torch 
import torch.optim as optim
from torch.utils.data import DataLoader
# SkLearn
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
## Datasets 
# Hate Xplain
hate_xplain = pd.read_csv("data/hate_xplain.csv")

# Implicit Hate 
implicit_hate = pd.read_csv('data/implicit_hate_v1_stg2_posts.tsv', delimiter='\t')
label_map = {
    'white_grievance': 0, 'incitement': 1, 'inferiority': 2,
    'irony': 3, 'stereotypical': 4, 'threatening': 5, 'other': 6
}
implicit_hate['class_label'] = implicit_hate['implicit_class'].map(label_map)
implicit_hate.drop("extra_implicit_class", axis=1, inplace=True)

In [3]:
hate_xplain = hate_xplain.sample(n=1000, random_state=42)
hate_xplain

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
2281,2326,3,0,3,0,1,934 8616\r\ni got a missed call from yo bitch
15914,16283,3,0,3,0,1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...
18943,19362,3,0,1,2,2,RT @eanahS__: @1inkkofrosess lol my credit ain...
16407,16780,3,0,3,0,1,RT @Maxin_Betha Wipe the cum out of them faggo...
13326,13654,3,1,2,0,1,Niggas cheat on they bitch and don't expect no...
...,...,...,...,...,...,...,...
2647,2710,3,0,3,0,1,@Bombfantasyyy @Angela_Mastr lmao angela you l...
13037,13357,3,0,3,0,1,"My favorite is #picslip that's a dumb bitch, w..."
5982,6153,3,1,2,0,1,"@freddurst shut up faggot, no one cares"
16621,17004,3,0,3,0,1,RT @Neonte: We was wearing shorts about a week...


In [4]:
def tokenize_data(texts, labels, tokenizer, max_length):
    if isinstance(texts, pd.Series):
        texts = texts.tolist()
    texts = [str(text) for text in texts] 

    if isinstance(labels, pd.Series):
        labels = labels.tolist()
    labels = torch.tensor(labels, dtype=torch.long)
    
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors="pt")
    dataset = torch.utils.data.TensorDataset(encodings["input_ids"], encodings["attention_mask"], labels)
    return dataset


In [ ]:
bert = 'bert-base-uncased'
bert_model = AutoModelForSequenceClassification.from_pretrained(
    bert, num_labels=3, 
    id2label={0: "hate speech", 1: "normal", 2: "offensive"}, 
    label2id={"hate speech": 0, "normal": 1, "offensive": 2}, max_length=128
    )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
hx_train_text, hx_test_text, hx_train_labels, hx_test_labels = train_test_split(hate_xplain['tweet'], hate_xplain['class'], test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained(bert, clean_up_tokenization_spaces=True)
hx_train = tokenize_data(hx_train_text, hx_train_labels, tokenizer, 512)
hx_test = tokenize_data(hx_test_text, hx_test_labels, tokenizer, 512)

hx_train_loader = DataLoader(hx_train, batch_size=16, shuffle=True)
hx_test_loader = DataLoader(hx_test, batch_size=16, shuffle=True)

In [7]:
hx_train_loader.__len__(), hx_test_loader.__len__()

(50, 13)

In [16]:
def train(model, data_loader, optimizer, epochs, device=None):
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train()

    for epoch in range(epochs):
        for input_ids, attention_mask, labels in data_loader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1} Loss: {loss.item()}")

def evaluate(model, data_loader, device=None):
    model.eval()
    model.to(device)

    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_preds, all_labels

In [9]:
optimizer = optim.AdamW(bert_model.parameters(), lr=5e-5)
train(bert_model, hx_train_loader, optimizer, 5, 'cuda')
pickle.dump(bert_model, open("BERT/bert.pkl", 'wb'))

Epoch 1 Loss: 0.6196829080581665
Epoch 2 Loss: 0.2246900349855423
Epoch 3 Loss: 0.21478521823883057
Epoch 4 Loss: 0.09045412391424179
Epoch 5 Loss: 0.05214119330048561


In [ ]:
hx_preds, hx_labels = evaluate(bert_model, hx_test_loader, 'cuda')
print(accuracy_score(hx_labels, hx_preds))

In [10]:
# train with all attention heads -> after training knock out attention heads at evaluation, at one head is more important than the other 
# attention heads -- overwrite model params to zeros -- output of attention head is zero'd out* -- little weird (average ?) 
#   set of weights for attention head -> knock out by (output is weighted sum) - intervene on output?, zero'd out keys weights 
## TODO: 1000 sampeles for training 
# transfer learning -> train on one dataset, evaluate on another dataset
#   *perhaps train on all 3 datasets, transfer on all 3 datasets*